<a href="https://colab.research.google.com/github/beaten-by-the-market/dart_disclosure/blob/main/disclosure_terms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 필요한 패키지 import

In [1]:
!pip install opendartreader
import pandas as pd
import OpenDartReader
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import time
from google.colab import userdata
from tqdm.notebook import tqdm

# 랭체인 관련
!pip install langchain-openai
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, RootModel
from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI

## OpenDart API를 통해서 공시 불러오기(오늘 공시)

In [2]:
#API KEY
from google.colab import userdata
opendart_api_key = userdata.get('opendart_personal')
openai_api_key = userdata.get('OPENAI_API_KEY')

# Opendart 연결
dart = OpenDartReader(opendart_api_key)

# 공시 불러오기
# 오늘 날짜를 기본으로 하여 불러옴
input_date = datetime.today().strftime('%Y-%m-%d')

# (공시유형) B  주요사항보고서 / I 거래소 수시공시
df_disclosure = dart.list(start=input_date, end=input_date, final=True, kind= ['B','I'])
print(input_date+' 공시 건수 : '+str(len(df_disclosure)))

# 새로운 칼럼 추가
df_disclosure['url'] = ''
df_disclosure['term'] = ''
df_disclosure['context'] = ''

2025-03-05 공시 건수 : 354


## 공시의 식별자(rceptno)에서 본문 URL 추출하는 함수 정의

In [3]:
# RCEPT_NO에서 본문 iframe을 불러오는 함수 생성
def get_second_url(rceptno):
  first_url = f"https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rceptno}"

  # 첫 번째 URL의 HTML 가져오기
  response = requests.get(first_url)
  if response.status_code != 200:
      raise Exception("Failed to fetch the page")

  # BeautifulSoup으로 HTML 파싱
  soup = BeautifulSoup(response.text, 'html.parser')

  # viewDoc 함수에서 필요한 값들 찾기
  script_tags = soup.find_all('script')
  extracted_values = None
  for script in script_tags:
      match = re.search(r'viewDoc\("(\d+)",\s*"(\d+)",\s*"(\d+)",\s*"(\d+)",\s*"(\d+)",\s*"([^"]+)"', script.text)
      if match:
          extracted_values = match.groups()  # ("rcpNo", "dcmNo", "eleId", "offset", "length", "dtd")
          break

  if not extracted_values:
      raise ValueError("viewDoc parameters not found in the page")

  # 추출된 값들을 URL에 맞게 매핑
  rcp_no, dcm_no, ele_id, offset, length, dtd = extracted_values

  # 두 번째 URL 생성
  second_url = f"https://dart.fss.or.kr/report/viewer.do?rcpNo={rcp_no}&dcmNo={dcm_no}&eleId={ele_id}&offset={offset}&length={length}&dtd={dtd}"

  return second_url

## 프롬프트 설정

In [4]:
# 각 용어 항목에 대해 term과 context만 포함된 Pydantic 모델 정의
class GlossaryEntry(BaseModel):
    term: str   # 용어
    context: str   # 용어가 등장하는 맥락

# JSON Output Schema 정의: 최종 출력은 GlossaryEntry 객체들의 리스트입니다.
class OutputSchema(RootModel[List[GlossaryEntry]]):
    pass

# PydanticOutputParser 생성
output_parser = PydanticOutputParser(pydantic_object=OutputSchema)

# Prompt template 정의
template = """
You are an AI assistant with expertise in analyzing financial disclosures and legal documents.
Your task is to extract glossary terms from the provided filing content. A glossary term is defined as any word or phrase that might be unclear to an investor who is not familiar with legal and regulatory terms such as 상법, 자본시장법, 공시규정, and 상장규정.

#### Instructions:
1. Read the provided content.
2. Identify any terms or phrases that an investor might search for to understand the filing better.
3. For each identified term, provide:
    - "term": the term or phrase as it appears.
    - "context": a brief description of where or how the term appears in the filing.
4. Output your result as a JSON list where each item is an object with the following structure:
```json
{{
  "term": "term1",
  "context": "context1"
}}
```
5. If no glossary terms are identified, output an empty list.
Example:
Input content: "이 공시에는 상법에 의거한 결산보고와 관련한 용어들이 포함되어 있으며, 자본시장법의 규정을 따르는 부분도 존재합니다."

Expected output: [ {{ "term": "상법", "context": "결산보고 관련 내용에서 등장" }}, {{ "term": "자본시장법", "context": "공시 내용 중 자본시장 규정 언급 부분" }} ]

Now, analyze the following content:
{content} """

#prompt_template = PromptTemplate.from_template(template)
# prompt_template 정의 부분 확인
prompt_template = PromptTemplate(template=template, input_variables=["content"])

model = ChatOpenAI( model="gpt-4o-mini", temperature=0, openai_api_key= openai_api_key)
chain = prompt_template | model | output_parser

## 조건걸기

In [5]:
# 우선 상위 10개만 해보기
df_backup = df_disclosure
df_disclosure = df_disclosure.head(10)

## 실행

In [6]:
# DataFrame을 순회하며 처리
for index, row in tqdm(df_disclosure.iterrows(), total=len(df_disclosure)):
    try:
        # second_url 생성
        second_url = get_second_url(row['rcept_no'])
        df_disclosure.at[index, 'url'] = second_url

        # 공시 내용 가져오기 (여기서는 간단히 처리)
        response = requests.get(second_url)
        content = BeautifulSoup(response.text, 'html.parser').get_text()

        # LangChain 실행
        output = chain.invoke({"content": content})

        if output and isinstance(output, OutputSchema):
            terms = [entry.term for entry in output.root]
            contexts = [entry.context for entry in output.root]
            df_disclosure.at[index, 'term'] = ', '.join(terms)
            df_disclosure.at[index, 'context'] = ', '.join(contexts)

    except Exception as e:
        print(f"Error processing row {index}: {e}")

    time.sleep(1)  # 1초 대기

# 결과 출력
df_disclosure

  0%|          | 0/10 [00:00<?, ?it/s]

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm,url,term,context
0,00125664,삼미금속,012210,N,주주총회소집결의,20250305601216,삼미금속,20250305,넥,https://dart.fss.or.kr/report/viewer.do?rcpNo=...,"주주총회, 감사보고, 영업보고, 내부회계관리제도, 외부감사인, 재무제표, 감사위원회...","주주총회소집 결의에서 주주총회의 일시, 장소, 의안 주요내용 등이 언급됨, 의안 주..."
1,00857480,사람인,143240,K,[기재정정]연결재무제표기준영업(잠정)실적(공정공시),20250305901215,사람인,20250305,코,https://dart.fss.or.kr/report/viewer.do?rcpNo=...,"연결재무제표, 정정신고, 외부감사인, 법인세비용차감전계속사업이익, 당기순이익, 지배...","재무제표 기준 영업(잠정)실적에서 언급됨, 정정 관련 공시서류에서 언급됨, 정정사유..."
2,01590139,우리카드이천이십일의일유동화전문유한회사,,E,주요사항보고서(해산사유발생),20250305000789,우리카드이천이십일의일유동화전문유한회사,20250305,,https://dart.fss.or.kr/report/viewer.do?rcpNo=...,"해산사유, 유동화증권, 정관, 상환, 청산 절차","정관 제34조 제2항에 의한 해산사유 발생에 대한 설명에서 등장, 유동화증권의 상환..."
3,01337017,이앤에치,341310,N,주주총회소집결의,20250305601151,이앤에치,20250305,넥,https://dart.fss.or.kr/report/viewer.do?rcpNo=...,"주주총회, 정기주주총회, 이사회결의일, 감사보고, 영업보고, 내부회계관리제도, 사내...","주주총회소집 결의에서 언급됨, 주주총회 구분에서 정기주주총회로 명시됨, 이사회결의일..."
4,00919966,신라젠,215600,K,주주총회집중일개최사유신고,20250305901211,신라젠,20250305,코,https://dart.fss.or.kr/report/viewer.do?rcpNo=...,"주주총회, 코스닥협회, 주주총회 분산 자율준수 프로그램, 결산, 외부감사인, 투표 기간","주주총회 집중일 개최 사유 신고에서 주주총회 개최와 관련된 내용, 주주총회 집중일과..."
5,01046391,싸이토젠,217330,K,조회공시요구(현저한시황변동)에대한답변(미확정),20250305901161,싸이토젠,20250305,코,https://dart.fss.or.kr/report/viewer.do?rcpNo=...,"조회공시요구, 현저한 시황변동, 코스닥 시장공시 규정, 임시주주총회, 주주제안, 이...","공시 제목 및 내용에서 현저한 시황변동에 대한 답변으로 언급됨, 조회공시 요구의 주..."
6,00862853,플레이디,237820,K,감사보고서제출,20250305901156,플레이디,20250305,코,https://dart.fss.or.kr/report/viewer.do?rcpNo=...,"감사보고서, 내부회계관리제도, 자본잠식률, 법인세비용차감전계속사업이익, 자기자본, ...","보고서 제출 및 감사의견과 관련된 내용에서 등장, 감사 의견과 관련된 부분에서 언급..."
7,00164742,현대자동차,005380,Y,감사보고서제출,20250305801167,현대자동차,20250305,유,https://dart.fss.or.kr/report/viewer.do?rcpNo=...,"감사보고서, 연결재무제표, 내부회계관리제도, 자본총계, 법인세비용차감전계속사업이익,...","보고서 제출 및 감사의견 관련 내용에서 등장, 지배회사 또는 지주회사의 연결재무제표..."
8,00302926,현대로템,064350,Y,감사보고서제출,20250305801159,현대로템,20250305,유,https://dart.fss.or.kr/report/viewer.do?rcpNo=...,"감사보고서, 연결재무제표, 내부회계관리제도, 자본총계, 부채총계, 매출액, 영업이익...","보고서 제출 및 감사의견과 관련된 내용에서 등장, 지배회사 또는 지주회사의 연결재무..."
9,00131054,유진증권,001200,Y,현금ㆍ현물배당결정,20250305801208,유진증권,20250305,유,https://dart.fss.or.kr/report/viewer.do?rcpNo=...,"결산배당, 현금배당, 차등배당, 시가배당율, 배당기준일, 주주총회, 이사회결의일, ...","배당구분에서 등장, 배당종류에서 등장, 차등배당 여부 항목에서 등장, 시가배당율(%..."
